In [4]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [5]:
import time
import textwrap

from langchain import hub
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings

# Load some data to summarize
loader = WebBaseLoader("https://teddylee777.github.io/data-science/optuna/")
docs = loader.load()
content = docs[0].page_content

# Get this prompt template
prompt = hub.pull("lawwu/chain_of_density")
# The chat model output is a JSON list of dicts, with SimpleJsonOutputParser
# we can convert it o a dict, and it suppors streaming.
json_parser = SimpleJsonOutputParser()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (50 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
"""
)
# Use only KOREAN language to reply."""

In [11]:
from langchain.llms import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.chat_models import ChatOllama
from langchain_huggingface.llms import HuggingFacePipeline
# llm = HuggingFaceEndpoint(
#     repo_id=os.environ["MODEL_ID"], 
#     max_new_tokens=2048,
#     temperature=0.1,
#     huggingfacehub_api_token=os.environ["HF_API_KEY"],
# )
# model = ChatHuggingFace(llm=llm)
model = ChatOllama(model="gemma:7b")
# llm = ChatOllama(model="mistral:7b")
# llm = HuggingFaceHub(
#     repo_id="beomi/Yi-Ko-6B",
#     model_kwargs={"temperature": 0.1, "max_length": 2048},
#     task="text-generation",
# )
# model = ChatHuggingFace(llm=llm)
# model = HuggingFacePipeline.from_model_id(
#     model_id="beomi/Yi-Ko-6B",
#     model_kwargs={"temperature": 0.1, "max_length": 1000000},
#     task="text-generation",
#     device=0,
# )

In [12]:
chain = (
    {"ARTICLE": RunnablePassthrough()}
    | prompt
    | model
    | json_parser
)

In [13]:
chain.invoke(content)

[{'Missing_Entities': 'AI assistant, search engine optimization, natural language processing',
  'Denser_Summary': 'This article explores the application of AI assistants in search engine optimization. It highlights the importance of natural language processing for effective interaction and the potential for AI-powered assistants to optimize website rankings and user engagement.'},
 {'Missing_Entities': 'data analysis, visualization, sentiment analysis',
  'Denser_Summary': 'The article emphasizes the role of data analysis in understanding user behavior and optimizing website content. It suggests the use of visualization and sentiment analysis techniques to gain insights from data and refine the search engine optimization strategy.'},
 {'Missing_Entities': 'competitive analysis, keyword research, backlink analysis',
  'Denser_Summary': 'The article discusses the importance of competitive analysis to identify industry trends and identify relevant keywords. It also highlights the signifi